### Original Kernel with minimal variations

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.set_printoptions(threshold=200)
pd.set_option('display.max_columns', 200)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../data/"))

# Any results you write to the current directory are saved as output.

['device_data.csv', 'pageviews_complemento.csv', 'sampleSubmission.csv', 'SITE_ID.csv', 'PAGE.csv', 'README.md', 'CONTENT_CATEGORY.csv', 'CONTENT_CATEGORY_BOTTOM.csv', 'pageviews.csv', 'conversiones.csv', 'CONTENT_CATEGORY_TOP.csv']


In [4]:
data = pd.concat([
       pd.read_csv("../data/pageviews.csv", parse_dates=["FEC_EVENT"]),
       pd.read_csv("../data/pageviews_complemento.csv", parse_dates=["FEC_EVENT"])
])

/home/horacio/.conda/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data.shape

In [ ]:
data.head()

### Benchmark Kernel

In [ ]:
X_test = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test = pd.concat(X_test, axis=1)

In [ ]:
X_test.head(5)

In [ ]:
X_test.shape

In [ ]:
X_test.iloc[0,0:65].sum() # la suma de las frecuencias debe ser 1

In [ ]:
data = data[data.FEC_EVENT.dt.month < 10]
X_train = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train = pd.concat(X_train, axis=1)

In [ ]:
X_train.sample(5)

In [ ]:
X_train.iloc[0,0:65].sum()

In [ ]:
features = list(set(X_train.columns).intersection(set(X_test.columns)))
X_train = X_train[features]
X_test = X_test[features]

In [ ]:
y_prev = pd.read_csv("../data/conversiones.csv")
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(set(X_train.index))
y_train.loc[list(idx)] = 1

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test = SelectKBest(chi2,k=250)
fit = test.fit(X_train, y_train)
X_train_alt = pd.DataFrame(fit.transform(X_train))
X_test_alt = pd.DataFrame(fit.transform(X_test))

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier

fi = []
test_probs = []
i = 0
for train_idx, valid_idx in model_selection.KFold(n_splits=10, shuffle=True).split(X_train):
    i += 1
    Xt = X_train.iloc[train_idx]
    yt = y_train.loc[X_train.index].iloc[train_idx]

    Xv = X_train.iloc[valid_idx]
    yv = y_train.loc[X_train.index].iloc[valid_idx]

    adalearner = AdaBoostClassifier()
    adalearner.fit(Xt, yt)
    adalearner.predict_proba(Xv)
    puntis = roc_auc_score(Xv,yv[1])
    
    test_probs.append(pd.Series(adalearner.predict_proba(X_test)[:,-1],
                            index=X_test.index, name="fold_" + str(i)))
    
    #learner = LGBMClassifier(n_estimators=10000)
    #learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
    #            eval_set=[(Xt, yt), (Xv, yv)])
    
test_probs.append(pd.Series(adalearner.predict_proba(X_test)[:, -1],
                            index=X_test.index))#, name="fold_" + str(i)))
fi.append(pd.Series(adalearner.feature_importances_ / adalearner.feature_importances_.sum(), index=X_train.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
test_probs.index.name="USER_ID"
test_probs.name="SCORE"
test_probs.to_csv("respuesta", header=True, compression="zip")
fi = pd.concat(fi, axis=1).mean(axis=1)

In [ ]:
dati = pd.read_csv("respuesta")

### Experimental models

#### Train set and Test set

We split the test set, so we use last three months to test.

In [5]:
data = data[data.FEC_EVENT.dt.month < 10]
X_test2 = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test2.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test2 = pd.concat(X_test2, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [6]:
data = data[(data.FEC_EVENT.dt.month < 7)]
X_train2 = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train2.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train2 = pd.concat(X_train2, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [7]:
y_prev2 = pd.read_csv("../data/conversiones.csv")
y_test2 = pd.Series(0, index=X_test2.index)
idx = set(y_prev2[y_prev2.mes >= 10].USER_ID.unique()).intersection(set(X_test2.index))
y_test2.loc[list(idx)] = 1

In [8]:
y_train2 = pd.Series(0, index=X_train2.index)
idx = set(y_prev2[(y_prev2.mes < 10) & (y_prev2.mes > 6)].USER_ID.unique()).intersection(set(X_train2.index))
y_train2.loc[list(idx)] = 1

In [9]:
print(X_train2.shape, y_train2.shape, X_test2.shape, y_test2.shape)

(11314, 1890) (11314,) (11529, 2026) (11529,)


In [10]:
features = list(set(X_train2.columns).intersection(set(X_test2.columns)))
X_train2 = X_train2[features]
X_test2 = X_test2[features]

In [ ]:
X_train2.shape

In [ ]:
# some feature selection

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test = SelectKBest(chi2,k=200)
fit = test.fit(X_train2, y_train2)
X_train2 = pd.DataFrame(fit.transform(X_train2))
X_test2 = pd.DataFrame(fit.transform(X_test2))

### Decision Tree Classifier

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

params = {
    'criterion' : ['gini','entropy'],
    'splitter' : ['best','random'],
    'min_samples_split' : [0.2,0.5,0.7,1.,2,3,4,5],
    'min_samples_leaf' : [1,3,5],
    'max_depth' : [None,1,2,4,6,8,10],
    'min_weight_fraction_leaf' : [0.,0.2,0.5],
    'max_features' : [None,10,25,50,75,100,150,200,250],
    'random_state' : [None,23,47,88],
    'max_leaf_nodes' : [None,10,25,50,100,200,500],
    'presort' : [False, True],
    'class_weight' : [None,'Balanced']
}

gs = GridSearchCV(dtc, params, scoring='roc_auc', cv=5)

In [ ]:
gs.fit(X_train2,y_train2)

In [ ]:
gs.best_score_

In [ ]:
gs.cv_results_

In [ ]:
from sklearn.metrics import roc_auc_score

scores = roc_auc_score(y_test2,parciales)

In [ ]:
scores

In [ ]:
print(X_train2.shape, X_test2.shape)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc2 = DecisionTreeClassifier()
dtc2.fit(X_train2, y_train2)
parciales = dtc2.predict(X_test2)
scores = roc_auc_score(y_test2,parciales)

scores

In [ ]:
# Optimization through GridSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

pg = [
    {"splitter":["best","random"],
     "max_depth":[1,2,3,4,5,7,10], 
     "max_features":[10,50,100,150,200], 
     "max_leaf_nodes":[10,50,100,150,200]}
]

opt = GridSearchCV(dtc2, pg, scoring="roc_auc",n_jobs=-1,cv=StratifiedKFold(5))
opt.fit(X_train2, y_train2)

In [ ]:
results = pd.DataFrame(opt.cv_results_)
results

In [ ]:
opt.best_score_

In [ ]:
optimised_decision_tree = opt.best_estimator_
optimised_decision_tree

In [ ]:
optimised_decision_tree.predict(X_test2)

In [ ]:
sol = optimised_decision_tree.predict_proba(X_test2)

In [ ]:
scores = roc_auc_score(y_test2,sol[:,1])
scores

### ADABOOST

In [ ]:
##### from sklearn.ensemble import AdaBoostClassifier

tree = DecisionTreeClassifier(max_depth = 4)
clf = AdaBoostClassifier(n_estimators = 8)
clf.fit(X_train2,y_train2)

In [ ]:
pred = clf.predict(X_test2)
pred

In [ ]:
probab = clf.predict_proba(X_test2)
probab

In [ ]:
scores = roc_auc_score(y_test2,probab[:,1])
scores